In [18]:
import requests

In [19]:
from bedrock.client.client import BedrockAPI

In [25]:
SERVER = "http://localhost:81/"

In [27]:
api = BedrockAPI(SERVER)

In [28]:
filepath =  '/home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv'

In [29]:
ingest_id = 'opals.spreadsheet.Spreadsheet.Spreadsheet'
resp = api.put_source('Rand2001',ingest_id, 'default',{'file':open(filepath, "rb")})
if resp.status_code == 201:
    print 'Source {0} successfully uploaded'.format(filepath)
else:
    print 'Failed to update {0}'.format(filepath)

INFO: 2017-09-01 16:51:33,099: putting source to: http://localhost:81/dataloader/sources/Rand2001/opals.spreadsheet.Spreadsheet.Spreadsheet/default/


Failed to update /home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv


In [30]:
available_sources = api.list("dataloader","sources").json()
source = (s for s in available_sources if s["name"] == "Rand2001").next()
source_id = source["src_id"]
endpoint = api.endpoint("dataloader","sources/%s/explore/" % source_id)
resp = requests.get(endpoint)

INFO: 2017-09-01 16:51:33,321: Listing: http://localhost:81/dataloader/sources/


In [31]:
resp = api.create_matrix(source_id, 'rand_mtx')
mtx = resp[0]
matrix_id = mtx['id']
print(mtx)
#resp

{u'src_id': u'2876d3ecda384e6ca195fefc0de078e9', u'name': u'rand_mtx', u'created': u'2017-09-01 20:51:33.753884', u'outputs': [u'features_original.txt', u'features.txt', u'matrix.csv'], u'id': u'ca9190c57c184c70aaf7007dcc76e97a', u'rootdir': u'/opt/bedrock/dataloader/data/2876d3ecda384e6ca195fefc0de078e9/ca9190c57c184c70aaf7007dcc76e97a/', u'filters': {u'fluid_dummy': {}, u'previous_decision': {}, u'group_size': {}, u'decision0d1c': {}, u'playerid': {}, u'_Icondition_4': {}, u'_Icondition_3': {}, u'_Icondition_2': {}, u'sessionnum': {}, u'num_neighbors': {}, u'round_num': {}, u'condition': {}}, u'mat_type': u'csv'}


In [32]:
resp = api.analytic('opals.stan.Stan.Stan_GLM')
if resp.json():
    print("Stan_GLM Opal Installed!")
else:
    print("Stan_GLM Opal Not Installed!")

Stan_GLM Opal Installed!


In [33]:
resp

<Response [200]>

In [36]:
analytic_id = "opals.select-from-dataframe.SelectByCondition.SelectByCondition"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
    {"attrname":"colname","value":"round_num"},
    {"attrname":"comparator","value":"=="},
    {"attrname":"value","value":"1"}
]

filtered_mtx = api.run_analytic(analytic_id, mtx, 'rand_round1_only', input_data=inputData, parameter_data=paramsData)

filtered_mtx

{u'analytic_id': u'opals.select-from-dataframe.SelectByCondition.SelectByCondition',
 u'created': u'2017-09-01 20:52:37.959536',
 u'id': u'47a3d3c16c23458483631938f182d161',
 u'name': u'rand_round1_only',
 u'outputs': [u'matrix.csv', u'features.txt'],
 u'parameters': [{u'attrname': u'colname', u'value': u'round_num'},
  {u'attrname': u'comparator', u'value': u'=='},
  {u'attrname': u'value', u'value': u'1'}],
 u'rootdir': u'/opt/bedrock/analytics/data/ca9190c57c184c70aaf7007dcc76e97a/47a3d3c16c23458483631938f182d161/',
 u'src_id': u'ca9190c57c184c70aaf7007dcc76e97a'}

In [40]:
analytic_id = "opals.stan.Stan.Stan_GLM"
inputData = {
    'matrix.csv': filtered_mtx,
    'features.txt': filtered_mtx
}

paramsData = [
            { "name" : "Formula", "attrname" : "formula", "value" : "decision0d1c ~ condition", "type" : "input" },
            { "name" : "GLM family", "attrname" : "family", "value" : 'binomial(link = "logit")', "type" : "input" },
            { "name" : "chains", "attrname" : "chains" , "value" : "3", "type" : "input"},
            { "name" : "iter", "attrname" : "iter" , "value" : "3000", "type" : "input"},
            { "name" : "prior", "attrname" : "prior" , "value" : "", "type" : "input"},
            { "name" : "prior_intercept", "attrname" : "prior_intercept" , "value" : "", "type" : "input"}
        ]

result_mtx = api.run_analytic(analytic_id, mtx, 'rand_stan_test', input_data=inputData, parameter_data=paramsData)

result_mtx

u'None\n'

## Get Stan_GLM outputs

In [12]:
prior_summary = api.download_results_matrix('rand_mtx', 'rand_stan_test', 'prior_summary.txt')
prior_summary



ERROR: 2017-09-01 16:48:11,052: Unknown remote file format


In [13]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table



TypeError: string indices must be integers

In [ ]:
source_name = 'Rand2011PNAS_cooperation_data'
matrix_id = 'rand_mtx'
matrix_name = 'rand_mtx'
feature_list = "sessionnum,condition,playerid,decision0d1c,previous_decision,round_num,num_neighbors,group_size,fluid_dummy,_Icondition_2,_Icondition_3,_Icondition_4".split(',')

In [ ]:
column_types = ['Numeric' for x in feature_list]
matrix_filters = dict.fromkeys(feature_list,{})
matboday = 